<a href="https://colab.research.google.com/github/HuynhThu04/GA_TSP/blob/main/A_star_nh%C3%A1p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-------------------------------------------------------------------------------
# Lớp biểu diễn các toạ độ (x, y)
#-------------------------------------------------------------------------------
class Position:
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y

#-------------------------------------------------------------------------------
# Lớp biểu diễn các ô
#-------------------------------------------------------------------------------
class Cell:
    # Constructor
    def __init__(self, position:(), parent:(), status, g, h, f):
        self.position = position # (x, y)
        self.parent   = parent   # parent (object) in a path
        self.status   = status   # {'free', 'start', 'dirty', 'clean'}

        self.g        = g # current   cost
        self.h        = h # estimated cost
        self.f        = f # full path cost
    #---------------------------------------------------------------------------

In [ ]:
import heapq
import random
from dataclasses import dataclass
from typing import List, Tuple, Optional, Set, FrozenSet

# -------------------------------------------------------------------------------
# Lớp biểu diễn các toạ độ (x, y)  (theo đề)
# -------------------------------------------------------------------------------
class Position:
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __iter__(self):
        yield self.x; yield self.y
    def __repr__(self):
        return f"({self.x},{self.y})"
    def __eq__(self, other):
        return isinstance(other, Position) and self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

# -------------------------------------------------------------------------------
# Lớp biểu diễn các ô (theo đề) – ở đây giữ để tương thích yêu cầu cấu trúc
# -------------------------------------------------------------------------------
class Cell:
    # Constructor
    def __init__(self, position:(), parent:(), status, g, h, f):
        self.position = position  # (x, y)
        self.parent   = parent    # parent (object) in a path
        self.status   = status    # {'free', 'start', 'dirty', 'clean'}
        self.g        = g         # current cost
        self.h        = h         # estimated cost
        self.f        = f         # full path cost

# ==========================
# Tiện ích hình học & MST
# ==========================
# 8 hướng → bước đi 1 đơn vị theo Chebyshev (max(|dx|,|dy|))
def chebyshev_dist(a: Tuple[int,int], b: Tuple[int,int]) -> int:
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def neighbors8(x: int, y: int, H: int, W: int):
    for dx in (-1,0,1):
        for dy in (-1,0,1):
            if dx == 0 and dy == 0:
                continue
            nx, ny = x+dx, y+dy
            if 0 <= nx < H and 0 <= ny < W:
                yield (nx, ny)

# MST Prim trên tập điểm với metric Chebyshev
def mst_length(points: List[Tuple[int,int]]) -> int:
    if not points or len(points) == 1:
        return 0
    N = len(points)
    used = [False]*N
    min_edge = [10**9]*N
    min_edge[0] = 0
    total = 0
    for _ in range(N):
        v = -1
        m = 10**9
        for i in range(N):
            if not used[i] and min_edge[i] < m:
                m = min_edge[i]; v = i
        used[v] = True
        total += m
        # update
        for u in range(N):
            if not used[u]:
                d = chebyshev_dist(points[v], points[u])
                if d < min_edge[u]:
                    min_edge[u] = d
    return total

# ==========================
# Trạng thái & Node cho A*
# ==========================
@dataclass(order=True)
class PQNode:
    f: int
    g: int
    t: int
    pos: Tuple[int,int]
    dirt: FrozenSet[Tuple[int,int]]
    parent: Optional['PQNode']
    action: Optional[str]  # "Move(x,y)" or "Suck"

def h_move(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]]) -> int:
    if not dirt:
        return 0
    # khoảng cách tới dirty gần nhất (Chebyshev)
    d_near = min(chebyshev_dist(pos, d) for d in dirt)
    # MST trên các dirty còn lại
    mst = mst_length(list(dirt))
    return d_near + mst

def h_suck(t: int, k: int) -> int:
    # t + (t+1) + ... + (t+k-1) = k*t + k*(k-1)/2
    return k*t + (k*(k-1))//2

def heuristic(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]], t: int) -> int:
    k = len(dirt)
    return h_move(pos, dirt) + h_suck(t, k)

# ==========================
# A* theo f(n) bạn đưa ra
# ==========================
def astar_clean(H: int, W: int, start: Tuple[int,int], dirt_init: Set[Tuple[int,int]]):
    start_dirt = frozenset(dirt_init)
    start_t = 0
    start_g = 0
    start_h = heuristic(start, start_dirt, start_t)
    start_node = PQNode(f=start_g+start_h, g=start_g, t=start_t, pos=start, dirt=start_dirt, parent=None, action=None)

    # open set (priority queue) & visited với best g theo trạng thái (pos, dirt, t)
    open_heap = [start_node]
    best_g = {(start, start_dirt, start_t): 0}

    while open_heap:
        cur = heapq.heappop(open_heap)

        # goal: không còn dirty
        if not cur.dirt:
            # reconstruct
            path = []
            node = cur
            while node and node.action is not None:
                path.append((node.action, node.pos, node.t, node.g))
                node = node.parent
            path.reverse()
            return path, cur.g

        # Sinh hành động Suck nếu đang đứng trên dirty
        if cur.pos in cur.dirt:
            new_dirt = set(cur.dirt); new_dirt.remove(cur.pos)
            new_dirt = frozenset(new_dirt)
            new_t = cur.t + 1
            new_g = cur.g + cur.t  # cost Suck tại thời điểm t hiện tại
            key = (cur.pos, new_dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(cur.pos, new_dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=cur.pos, dirt=new_dirt,
                    parent=cur, action="Suck"
                ))

        # Sinh các hành động Move (8 hướng)
        x, y = cur.pos
        for nx, ny in neighbors8(x, y, H, W):
            new_pos = (nx, ny)
            new_t = cur.t + 1
            new_g = cur.g + 1  # cost Move = 1
            key = (new_pos, cur.dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(new_pos, cur.dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=new_pos, dirt=cur.dirt,
                    parent=cur, action=f"Move{new_pos}"
                ))

    return None, None  # không tìm được (trường hợp hiếm)

# ==========================
# Tạo bài toán ngẫu nhiên & chạy
# ==========================
def make_random_problem(H: int, W: int, k_dirty: int, seed: Optional[int] = None, start: Optional[Tuple[int,int]] = None):
    rng = random.Random(seed)
    if start is None:
        start = (rng.randrange(H), rng.randrange(W))
    all_cells = [(i,j) for i in range(H) for j in range(W) if (i,j) != start]
    rng.shuffle(all_cells)
    dirties = set(all_cells[:max(0, min(k_dirty, H*W - 1))])
    return start, dirties

def run(H=5, W=7, k_dirty=6, seed=0):
    start, dirties = make_random_problem(H, W, k_dirty, seed=seed)
    path, total_cost = astar_clean(H, W, start, dirties)
    print(f"Grid size: {H}x{W}")
    print(f"Start: {start}")
    print(f"Dirty cells: {sorted(dirties)}")
    if path is None:
        print("Không tìm được lời giải.")
        return
    print("\nChuỗi hành động (action, position, t, g tích lũy):")
    for step_idx, (act, pos, t, g) in enumerate(path, 1):
        print(f"{step_idx:3d}. {act:>10}  -> pos={pos}  t={t}  g={g}")
    print(f"\nTổng chi phí cuối cùng: {total_cost}")

# Ví dụ chạy thử:
if __name__ == "__main__":
    run(H=6, W=8, k_dirty=8, seed=42)


Grid size: 6x8
Start: (5, 1)
Dirty cells: [(0, 4), (1, 1), (1, 3), (2, 3), (2, 7), (3, 1), (3, 2), (5, 5)]

Chuỗi hành động (action, position, t, g tích lũy):
  1. Move(4, 0)  -> pos=(4, 0)  t=1  g=1
  2. Move(3, 1)  -> pos=(3, 1)  t=2  g=2
  3.       Suck  -> pos=(3, 1)  t=3  g=4
  4. Move(3, 2)  -> pos=(3, 2)  t=4  g=5
  5.       Suck  -> pos=(3, 2)  t=5  g=9
  6. Move(2, 3)  -> pos=(2, 3)  t=6  g=10
  7.       Suck  -> pos=(2, 3)  t=7  g=16
  8. Move(1, 3)  -> pos=(1, 3)  t=8  g=17
  9.       Suck  -> pos=(1, 3)  t=9  g=25
 10. Move(0, 4)  -> pos=(0, 4)  t=10  g=26
 11.       Suck  -> pos=(0, 4)  t=11  g=36
 12. Move(0, 5)  -> pos=(0, 5)  t=12  g=37
 13. Move(1, 6)  -> pos=(1, 6)  t=13  g=38
 14. Move(2, 7)  -> pos=(2, 7)  t=14  g=39
 15.       Suck  -> pos=(2, 7)  t=15  g=53
 16. Move(3, 6)  -> pos=(3, 6)  t=16  g=54
 17. Move(4, 5)  -> pos=(4, 5)  t=17  g=55
 18. Move(5, 5)  -> pos=(5, 5)  t=18  g=56
 19.       Suck  -> pos=(5, 5)  t=19  g=74
 20. Move(4, 4)  -> pos=(4, 4)  t=20  

In [ ]:
#-------------------------------------------------------------------------------
# Lớp biểu diễn các toạ độ (x, y)
#-------------------------------------------------------------------------------
class Position:
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __iter__(self):
        yield self.x; yield self.y
    def __repr__(self):
        return f"({self.x},{self.y})"
    def __eq__(self, other):
        return isinstance(other, Position) and self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

#-------------------------------------------------------------------------------
# Lớp biểu diễn các ô
#-------------------------------------------------------------------------------
class Cell:
    # Constructor
    def __init__(self, position:(), parent:(), status, g, h, f):
        self.position = position # (x, y)
        self.parent   = parent   # parent (object) in a path
        self.status   = status   # {'free', 'start', 'dirty', 'clean'}

        self.g        = g # current   cost
        self.h        = h # estimated cost
        self.f        = f # full path cost
    #---------------------------------------------------------------------------

In [ ]:
import heapq
import random
from dataclasses import dataclass
from typing import List, Tuple, Optional, Set, FrozenSet

# ==========================
# Tiện ích hình học & MST
# ==========================
# 8 hướng → bước đi 1 đơn vị theo Chebyshev (max(|dx|,|dy|))
def chebyshev_dist(a: Tuple[int,int], b: Tuple[int,int]) -> int:
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def neighbors8(x: int, y: int, H: int, W: int):
    for dx in (-1,0,1):
        for dy in (-1,0,1):
            if dx == 0 and dy == 0:
                continue
            nx, ny = x+dx, y+dy
            if 0 <= nx < H and 0 <= ny < W:
                yield (nx, ny)

# MST Prim trên tập điểm với metric Chebyshev
def mst_length(points: List[Tuple[int,int]]) -> int:
    if not points or len(points) == 1:
        return 0
    N = len(points)
    used = [False]*N
    min_edge = [10**9]*N
    min_edge[0] = 0
    total = 0
    for _ in range(N):
        v = -1
        m = 10**9
        for i in range(N):
            if not used[i] and min_edge[i] < m:
                m = min_edge[i]; v = i
        used[v] = True
        total += m
        # update
        for u in range(N):
            if not used[u]:
                d = chebyshev_dist(points[v], points[u])
                if d < min_edge[u]:
                    min_edge[u] = d
    return total

# ==========================
# Trạng thái & Node cho A*
# ==========================
@dataclass(order=True)
class PQNode:
    f: int
    g: int
    t: int
    pos: Tuple[int,int]
    dirt: FrozenSet[Tuple[int,int]]
    parent: Optional['PQNode']
    action: Optional[str]  # "Move(x,y)" or "Suck"

def h_move(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]]) -> int:
    if not dirt:
        return 0
    # khoảng cách tới dirty gần nhất (Chebyshev)
    d_near = min(chebyshev_dist(pos, d) for d in dirt)
    # MST trên các dirty còn lại
    mst = mst_length(list(dirt))
    return d_near + mst

def h_suck(t: int, k: int) -> int:
    # t + (t+1) + ... + (t+k-1) = k*t + k*(k-1)/2
    return k*t + (k*(k-1))//2

def heuristic(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]], t: int) -> int:
    k = len(dirt)
    return h_move(pos, dirt) + h_suck(t, k)

# ==========================
# A* theo f(n) bạn đưa ra
# ==========================
def astar_clean(H: int, W: int, start: Tuple[int,int], dirt_init: Set[Tuple[int,int]]):
    start_dirt = frozenset(dirt_init)
    start_t = 0
    start_g = 0
    start_h = heuristic(start, start_dirt, start_t)
    start_node = PQNode(f=start_g+start_h, g=start_g, t=start_t, pos=start, dirt=start_dirt, parent=None, action=None)

    # open set (priority queue) & visited với best g theo trạng thái (pos, dirt, t)
    open_heap = [start_node]
    best_g = {(start, start_dirt, start_t): 0}

    while open_heap:
        cur = heapq.heappop(open_heap)

        # goal: không còn dirty
        if not cur.dirt:
            # reconstruct
            path = []
            node = cur
            while node and node.action is not None:
                path.append((node.action, node.pos, node.t, node.g))
                node = node.parent
            path.reverse()
            return path, cur.g

        # Sinh hành động Suck nếu đang đứng trên dirty
        if cur.pos in cur.dirt:
            new_dirt = set(cur.dirt); new_dirt.remove(cur.pos)
            new_dirt = frozenset(new_dirt)
            new_t = cur.t + 1
            new_g = cur.g + cur.t  # cost Suck tại thời điểm t hiện tại
            key = (cur.pos, new_dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(cur.pos, new_dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=cur.pos, dirt=new_dirt,
                    parent=cur, action="Suck"
                ))

        # Sinh các hành động Move (8 hướng)
        x, y = cur.pos
        for nx, ny in neighbors8(x, y, H, W):
            new_pos = (nx, ny)
            new_t = cur.t + 1
            new_g = cur.g + 1  # cost Move = 1
            key = (new_pos, cur.dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(new_pos, cur.dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=new_pos, dirt=cur.dirt,
                    parent=cur, action=f"Move{new_pos}"
                ))

    return None, None  # không tìm được (trường hợp hiếm)

# ==========================
# Tạo bài toán ngẫu nhiên & chạy
# ==========================
def make_random_problem(H: int, W: int, k_dirty: int, seed: Optional[int] = None, start: Optional[Tuple[int,int]] = None):
    rng = random.Random(seed)
    if start is None:
        start = (rng.randrange(H), rng.randrange(W))
    all_cells = [(i,j) for i in range(H) for j in range(W) if (i,j) != start]
    rng.shuffle(all_cells)
    dirties = set(all_cells[:max(0, min(k_dirty, H*W - 1))])
    return start, dirties

def run(H=5, W=7, k_dirty=6, seed=0):
    start, dirties = make_random_problem(H, W, k_dirty, seed=seed)
    path, total_cost = astar_clean(H, W, start, dirties)
    print(f"Grid size: {H}x{W}")
    print(f"Start: {start}")
    print(f"Dirty cells: {sorted(dirties)}")
    if path is None:
        print("Không tìm được lời giải.")
        return
    print("\nChuỗi hành động (action, position, t, g tích lũy):")
    for step_idx, (act, pos, t, g) in enumerate(path, 1):
        print(f"{step_idx:3d}. {act:>10}  -> pos={pos}  t={t}  g={g}")
    print(f"\nTổng chi phí cuối cùng: {total_cost}")

In [ ]:
#-------------------------------------------------------------------------------
# Lớp biểu diễn các toạ độ (x, y)
#-------------------------------------------------------------------------------
class Position:
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __iter__(self):
        yield self.x; yield self.y
    def __repr__(self):
        return f"({self.x},{self.y})"
    def __eq__(self, other):
        return isinstance(other, Position) and self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

#-------------------------------------------------------------------------------
# Lớp biểu diễn các ô
#-------------------------------------------------------------------------------
class Cell:
    # Constructor
    def __init__(self, position:(), parent:(), status, g, h, f):
        self.position = position # (x, y)
        self.parent   = parent   # parent (object) in a path
        self.status   = status   # {'free', 'start', 'dirty', 'clean'}

        self.g        = g # current   cost
        self.h        = h # estimated cost
        self.f        = f # full path cost
    #---------------------------------------------------------------------------

In [ ]:
import heapq
import random
from dataclasses import dataclass
from typing import List, Tuple, Optional, Set, FrozenSet

# ==========================
# Tiện ích hình học & MST
# ==========================
# 8 hướng → bước đi 1 đơn vị theo Chebyshev (max(|dx|,|dy|))
def chebyshev_dist(a: Tuple[int,int], b: Tuple[int,int]) -> int:
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def neighbors8(x: int, y: int, H: int, W: int):
    for dx in (-1,0,1):
        for dy in (-1,0,1):
            if dx == 0 and dy == 0:
                continue
            nx, ny = x+dx, y+dy
            if 0 <= nx < H and 0 <= ny < W:
                yield (nx, ny)

# MST Prim trên tập điểm với metric Chebyshev
def mst_length(points: List[Tuple[int,int]]) -> int:
    if not points or len(points) == 1:
        return 0
    N = len(points)
    used = [False]*N
    min_edge = [10**9]*N
    min_edge[0] = 0
    total = 0
    for _ in range(N):
        v = -1
        m = 10**9
        for i in range(N):
            if not used[i] and min_edge[i] < m:
                m = min_edge[i]; v = i
        used[v] = True
        total += m
        # update
        for u in range(N):
            if not used[u]:
                d = chebyshev_dist(points[v], points[u])
                if d < min_edge[u]:
                    min_edge[u] = d
    return total

# ==========================
# Trạng thái & Node cho A*
# ==========================
@dataclass(order=True)
class PQNode:
    f: int
    g: int
    t: int
    pos: Tuple[int,int]
    dirt: FrozenSet[Tuple[int,int]]
    parent: Optional['PQNode']
    action: Optional[str]  # "Move(x,y)" or "Suck"

def h_move(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]]) -> int:
    if not dirt:
        return 0
    # khoảng cách tới dirty gần nhất (Chebyshev)
    d_near = min(chebyshev_dist(pos, d) for d in dirt)
    # MST trên các dirty còn lại
    mst = mst_length(list(dirt))
    return d_near + mst

def h_suck(t: int, k: int) -> int:
    # t + (t+1) + ... + (t+k-1) = k*t + k*(k-1)/2
    return k*t + (k*(k-1))//2

def heuristic(pos: Tuple[int,int], dirt: FrozenSet[Tuple[int,int]], t: int) -> int:
    k = len(dirt)
    return h_move(pos, dirt) + h_suck(t, k)

# ==========================
# A* theo f(n) bạn đưa ra
# ==========================
def astar_clean(H: int, W: int, start: Tuple[int,int], dirt_init: Set[Tuple[int,int]]):
    start_dirt = frozenset(dirt_init)
    start_t = 0
    start_g = 0
    start_h = heuristic(start, start_dirt, start_t)
    start_node = PQNode(f=start_g+start_h, g=start_g, t=start_t, pos=start, dirt=start_dirt, parent=None, action=None)

    # open set (priority queue) & visited với best g theo trạng thái (pos, dirt, t)
    open_heap = [start_node]
    best_g = {(start, start_dirt, start_t): 0}

    while open_heap:
        cur = heapq.heappop(open_heap)

        # goal: không còn dirty
        if not cur.dirt:
            # reconstruct
            path = []
            node = cur
            while node and node.action is not None:
                path.append((node.action, node.pos, node.t, node.g))
                node = node.parent
            path.reverse()
            return path, cur.g

        # Sinh hành động Suck nếu đang đứng trên dirty
        if cur.pos in cur.dirt:
            new_dirt = set(cur.dirt); new_dirt.remove(cur.pos)
            new_dirt = frozenset(new_dirt)
            new_t = cur.t + 1
            new_g = cur.g + cur.t  # cost Suck tại thời điểm t hiện tại
            key = (cur.pos, new_dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(cur.pos, new_dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=cur.pos, dirt=new_dirt,
                    parent=cur, action="Suck"
                ))

        # Sinh các hành động Move (8 hướng)
        x, y = cur.pos
        for nx, ny in neighbors8(x, y, H, W):
            new_pos = (nx, ny)
            new_t = cur.t + 1
            new_g = cur.g + 1  # cost Move = 1
            key = (new_pos, cur.dirt, new_t)
            if best_g.get(key, 10**18) > new_g:
                best_g[key] = new_g
                h = heuristic(new_pos, cur.dirt, new_t)
                heapq.heappush(open_heap, PQNode(
                    f=new_g + h, g=new_g, t=new_t, pos=new_pos, dirt=cur.dirt,
                    parent=cur, action=f"Move{new_pos}"
                ))

    return None, None  # không tìm được (trường hợp hiếm)

# ==========================
# Tạo bài toán ngẫu nhiên & chạy
# ==========================
def make_random_problem(H: int, W: int, k_dirty: int, seed: Optional[int] = None, start: Optional[Tuple[int,int]] = None):
    rng = random.Random(seed)
    if start is None:
        start = (rng.randrange(H), rng.randrange(W))
    all_cells = [(i,j) for i in range(H) for j in range(W) if (i,j) != start]
    rng.shuffle(all_cells)
    dirties = set(all_cells[:max(0, min(k_dirty, H*W - 1))])
    return start, dirties

def run(H=5, W=7, k_dirty=6, seed=0):
    start, dirties = make_random_problem(H, W, k_dirty, seed=seed)
    path, total_cost = astar_clean(H, W, start, dirties)
    print(f"Grid size: {H}x{W}")
    print(f"Start: {start}")
    print(f"Dirty cells: {sorted(dirties)}")
    if path is None:
        print("Không tìm được lời giải.")
        return
    print("\nChuỗi hành động (action, position, t, g tích lũy):")
    for step_idx, (act, pos, t, g) in enumerate(path, 1):
        print(f"{step_idx:3d}. {act:>10}  -> pos={pos}  t={t}  g={g}")
    print(f"\nTổng chi phí cuối cùng: {total_cost}")